### Day 15 Part 1

My first attempt incorrectly assumed we could only go down or to the right, which is not the case. It managed to solve Part 1 & 2 for test data and Part 1 for actual, but ended there.

Did some reading and will be testing `dijkstra algo`: 
- https://www.programiz.com/dsa/dijkstra-algorithm
- https://stackabuse.com/dijkstras-algorithm-in-python/


### General Process:

- set all costs to a very large number (infinity) except for starting point (set to 0)
- find the coordinate with the smallest path (referencing a queue here)
    - this will be the starting point at the beginning
- find all neighbors of this coordinate:
    - In my case below this is left, right, up or down. 
    - we then add coordinate to a set marking it visited (this way we don't backtrack)
- iterate over the neighbors:
    - check to ensure neighbor hasn't been visited already (prevents unecessary searches)
    - determine the cost to go from coordinate to neighbor
    - if this cost is less than the current neigbor cost, then overwrite the value in the cost dictionary and add the neighbor to the priority queue 
- restart process with next shortest path
    - process is run until all nodes have been visited. 
    - i assume there is a way to make this greedy (e.g. stop process once a path to the end has been found), but that wouldn't guarantee shortest distance.
    
    
### Learnings:

- For storing visited vertices I started by using a list, which was INSANELY slow on Part 2 of actual data. I switched to a set and was solved in 1.3 seconds, just massive differential and reminder of how slow lists are. A set worked perfectly for this need (storing visited nodes, no mutations needed)

- Priority Queue:
    - Going to prioritize an addition based on the cost. Specifically, default is to prioritize with lowest value (shortest path in our case)
    - for this algorithm the use of this data structure allows us to minimize search time for the next closest distance (smallest risk in our case). 
    - This is critical as we start with all points set to infinity risk aside from the start. When we then need to determine where to go next it makes sense to use smallest cost/ risk, which is where priority queue is nice.
        - Secondly, we just keep adding the shortest routes to a specific point....overtime the priority queue could contain the same coord multiple times, but will prioritize the shortest route (lowest risk value) 

In [1]:
import time
import numpy as np
from itertools import accumulate 
from queue import PriorityQueue

# read data 
with open('data/day15_test.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]
    
# then break down for each list into an array
single_list = []
for row in data:
    single_list.extend([int(x) for x in row])
    
# build matrix
mat = np.asarray(single_list)
mat.shape = (len(data), len(data[0]))

In [2]:
# priority queue example: lowest cost is prioritized, even though it was added in middle
pq = PriorityQueue()
pq.put((100, 'a'))
pq.put((10, 'b'))
pq.put((50, 'c'))
print(pq.get())
print(pq.get())
print(pq.get())

(10, 'b')
(50, 'c')
(100, 'a')


In [3]:
# New functions
def findNeighbors(matrix, coord):
    """Return neighboring coordinates within boundary of matrix"""
    i = coord[0]
    j = coord[1]
    
    r_max = matrix.shape[0]
    c_max = matrix.shape[1]
    
    coord_list = [(i, j+1), (i, j-1), (i-1,j), (i+1,j)]
    final_list = []

    for coord in coord_list:
        i = coord[0]
        j = coord[1]
        if (i >= 0 and i < r_max) and (j >= 0 and j < c_max):
            final_list.append(coord)
    return final_list

def dijkstra(matrix, start):
    """Dijkstra Algorithm 
    Source: https://stackabuse.com/dijkstras-algorithm-in-python/
    Slightly modified for my purposes
    """
    D = {} # key is coord, val is the lowest cost to get to this coord from start
    visited = set() # store coords we have viewed
    
    # set all vals to infinity
    for r in range(matrix.shape[0]):
        for c in range(matrix.shape[1]):
            D[(r,c)] = float('inf')
    
    # start is 0 cost
    D[start] = 0
    
    # priority queue will keep shortest distances at the top
    pq = PriorityQueue()
    pq.put((0, start))
    
    while not pq.empty():
        (dist, current_coord) = pq.get() # get shortest path coord, pops off top of queue
        visited.add(current_coord) # add to visited node, we don't need to review it again
        
        # determine neighbors to check
        neighbors = findNeighbors(matrix, current_coord)
        
        # iterate through neighbors to determine if current_coord -> neighbor is shorter path than
        # current path to neighbor
        for neighbor in neighbors:
            distance = matrix[neighbor] # distance from current coord -> neighbor
            if neighbor not in visited:
                old_cost = D[neighbor] # current cost for start -> neighbor
                new_cost = D[current_coord] + distance # new cost for start -> current coord -> neighbor
                if new_cost < old_cost:
                    pq.put((new_cost, neighbor)) # add to queue, will rise based on how lost cost is
                    D[neighbor] = new_cost # override current smallest cost for neighbor coord
    return D

In [4]:
# test data
paths = dijkstra(mat, (0,0))
paths[mat.shape[0] -1,mat.shape[0] -1]

40

In [5]:
# read data 
with open('data/day15.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]
    
# then break down for each list into an array
single_list = []
for row in data:
    single_list.extend([int(x) for x in row])
    
# build matrix
mat = np.asarray(single_list)
mat.shape = (len(data), len(data[0]))
print(mat.shape)
paths = dijkstra(mat, (0,0))
print(f"Total risk: {[mat.shape[0] -1,mat.shape[0] -1]}")

(100, 100)
Total risk: [99, 99]


### Part 2: Test Case

The above solution should work, but need to handle the expansion.

In [6]:
# read data 
with open('data/day15_test.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]
    
# then break down for each list into an array
single_list = []
for row in data:
    single_list.extend([int(x) for x in row])
    
# build matrix
mat = np.asarray(single_list)
mat.shape = (len(data), len(data[0]))

In [7]:
def matrixAdd(mat, idx, col=True):
    """Properly add last matrix and return stacked"""
    i,i2,j,j2 = idx
    add_mat = mat[i:i2,j:j2] + 1
    add_mat = np.where(add_mat > 9, 1, add_mat)
    
    if col:
        return np.column_stack((mat,add_mat))
    return np.row_stack((mat,add_mat))

In [8]:
# Build out top row:
a = 0
b = 10
for i in range(4):
    mat = matrixAdd(mat, (0,10,a,b))
    a += 10
    b += 10
test = [int(x) for x in '11637517422274862853338597396444961841755517295286']

# confirm top row transormed properly 
assert(list(mat[0,:]) == test)

In [9]:
# Build out second row - pretty easy now
a = 0
b = 10
for _ in range(4):
    mat = matrixAdd(mat, (a,b,0,50), col = False)
    a += 10
    b += 10

# confirm second row transormed properly 
test = [int(x) for x in '22748628533385973964449618417555172952866628316397']
assert(list(mat[10,:]) == test)
assert(mat.shape == (50,50))
s = time.time()
paths = dijkstra(mat, (0,0))
print(f"Total time {time.time()-s}")
assert(paths[mat.shape[0] -1,mat.shape[0] -1] == 315)

Total time 0.013150930404663086


### Part 2: Actual Data

For some reason not working

In [10]:
# read data 
with open('data/day15.txt') as fh:
    data = [line.strip('\n') for line in fh.readlines()]
    
# then break down for each list into an array
single_list = []
for row in data:
    single_list.extend([int(x) for x in row])
    
# build matrix
mat = np.asarray(single_list)
mat.shape = (len(data), len(data[0]))
print(f"Matrix shape: {mat.shape}")

Matrix shape: (100, 100)


In [11]:
# Build out top row:
a = 0
b = 100
for i in range(4):
    mat = matrixAdd(mat, (0,100,a,b))
    a += 100
    b += 100

# Build out rows 2-5
a = 0
b = 100
for _ in range(4):
    mat = matrixAdd(mat, (a,b,0,500), col = False)
    a += 100
    b += 100

# confirm values increment properly
test = []
for i in [0,100,200,300,400]:
    for j in [0,100,200,300,400]:
        test.append(mat[i, j])
    print(test)
    test = []

# Solve for shortest path
s = time.time()
paths = dijkstra(mat, (0,0))
print(time.time() - s)
print(f"Shortest path to end: {paths[mat.shape[0] -1,mat.shape[0] -1]} risk")

[1, 2, 3, 4, 5]
[2, 3, 4, 5, 6]
[3, 4, 5, 6, 7]
[4, 5, 6, 7, 8]
[5, 6, 7, 8, 9]
1.346792221069336
Shortest path to end: 2844 risk
